In [15]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, accuracy_score,f1_score,mean_absolute_error
from sklearn.impute import SimpleImputer
import xgboost 
from sklearn.linear_model import LogisticRegression
import pickle as pk

In [16]:
diabetesdataurl='diabetes.csv'

diabetesdata= pd.read_csv(diabetesdataurl)

In [17]:
diabetesdata[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']] = diabetesdata[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']].replace(0,np.NaN)

In [18]:
diabetesdata.dropna(inplace=True)

In [19]:
X=diabetesdata.drop(['Outcome'],axis=1)

In [20]:
Y=diabetesdata['Outcome']

In [21]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=42)


In [22]:
classifier=xgboost.XGBClassifier(learning_rate =0.0001,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
# classifier=LogisticRegression()
# from sklearn import svm
# classifier=svm.SVC(kernel='linear')
classifier.fit(X_train,Y_train)
pred=classifier.predict(X_train)

C:\Users\VICKFURY\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\VICKFURY\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[06:52:23] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [23]:
trainaccuracy=accuracy_score(Y_train,pred)
f1score= f1_score(Y_train,pred, average=None)
trainaccuracy, f1score

(0.9137380191693291, array([0.93556086, 0.86956522]))

In [24]:
preds=classifier.predict(X_test)
preds

array([1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], dtype=int64)

In [25]:
testaccuracy=accuracy_score(Y_test,preds)
testaccuracy

0.8227848101265823

In [26]:
# sample1=[6,148,72,35,0,33.6,0.627,50]
sample1=[5,116,74,0,0,25.6,0.201,30]
sample1=np.array(sample1)
sample1=sample1.reshape(1,-1)
# sample2=scaler.transform(sample1)

finalpred=classifier.predict(sample1)
if finalpred==1:
    print ('You have diabetes')
else:
    print ('No diabetes')

No diabetes


In [27]:
ml= "diabetespredmodelusingxgboost.pkl"
md=pk.dump(classifier,open(ml,'wb'))

In [28]:
model =pk.load(open(ml,'rb'))
model

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8,
              enable_categorical=False, gamma=0, gpu_id=-1,
              importance_type=None, interaction_constraints='',
              learning_rate=0.0001, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=1000, n_jobs=4, nthread=4, num_parallel_tree=1,
              predictor='auto', random_state=27, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=1, seed=27, subsample=0.8, tree_method='exact',
              validate_parameters=1, verbosity=None)